# Решение спитцеровского уравнения теплопроводности плазмы

## Неявная разностная схема с нелинейностью на нижнем слое

Для численного решения задачи будем использовать неявную разностную схему с нелинейностью на нижнем слое

$$
\frac{y^{n+1}_{m} - y^{n}_{m}}{\tau} = \frac{1}{h}\left[k^n_{m + 1/2}\frac{y^{n+1}_{m+1} - y^{n+1}_{m}}{h} - k^n_{m - 1/2}\frac{y^{n+1}_{m} - y^{n+1}_{m - 1}}{h}\right] + f^n_m
$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from matplotlib.pyplot import axes
#from celluloid import Camera

In [2]:
def u_init(x, To):
    u = np.zeros(x.size)
    for i in range(len(x)):
        if x[i] <= 1:
            u[i] = To
    return u

### Метод прогонки

In [3]:
import utils

In [ ]:
A = np.array([[11,4,0,0],
              [3,10,4,0],
              [0,1,7,5],
              [0,0,3,4]])
a = np.diag(A, -1)
b = np.diag(A)
c = np.diag(A, 1)
d = np.array([3, 4, 5, 6.])
print(np.linalg.solve(A, d))
print(utils.thomas_solver(a, -b, c, d))

#### Применение метода

In [ ]:
# Начальные значения
a = 0
b = 2
t_0 = 0
T = 0.001
To = 2

In [ ]:
# Число узлов
N = 6000
M = 400

# Размер шага сетки
h = (b - a) / M
tau = (T - t_0) / N
print(f'{h=}, {tau=: .6f}')

print(f'sigma = {tau / h ** 2 * 0.5: .4f}')

In [ ]:
t = np.linspace(t_0, T, N)
x = np.linspace(a, b, M)

In [ ]:
u1 = np.zeros((N + 1, M), dtype=np.double)
u1[0, :] = u_init(x, 2)

u2 = np.zeros((N + 1, M), dtype=np.double)
u2[0, :] = u_init(x, 2)

In [ ]:
for n in tqdm(range(N)):
    a_e, b_e, c_e, d_e = utils.make_diagonals(u1, u2, n, tau, h, M, mode='electrons')
    u_sol = utils.thomas_solver(a_e, b_e, c_e, d_e)
    u1[n + 1, :] = u_sol
    a_i, b_i, c_i, d_i = utils.make_diagonals(u1, u2, n, tau, h, M, mode='ion')
    u2[n + 1, :] = utils.thomas_solver(a_i, b_i, c_i, d_i)

#### Визуализация

In [ ]:
ax = axes(xlim=(0.9, 1.1), ylim=(0, 2.5))
for i in range(6):
    ax.plot(x, u1[i, :])
    #ax.plot(x, u2[i, :], color="red")

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import axes
from celluloid import Camera

fig = plt.figure()
camera = Camera(fig)
ax = axes(xlim=(0, b), ylim=(0, 2.5))

for i in range(0, N, 20):
    ax.plot(x, u1[i, :], color="blue")
    ax.plot(x, u2[i, :], color="red")
    camera.snap()
animation = camera.animate()
animation.save('oh_my.gif', writer = 'imagemagick')

In [ ]:
from IPython.display import Image
Image(url='oh_my.gif')

In [18]:
from IPython.display import Image
Image(url='oh_my.gif')

### Метод потоковой прогонки